In [1]:
import xgboost as xgb

xgb_model = xgb.Booster()
xgb_model.load_model('xgb_model_log.model')

In [5]:
import pandas as pd
import numpy as np

# ["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트시가_log", "영업_개월_평균", "폐업_개월_평균", "매출_건수", '업종_명_가전제품_및_통신기기_도소매업',
       #'업종_명_개인_서비스업', '업종_명_교육_및_사업자원_서비스업', '업종_명_기타_도소매업', '업종_명_기타_서비스업',
       #'업종_명_부동산_및_임대업', '업종_명_생활용품_도소매업', '업종_명_수리업', '업종_명_숙박업',
       #'업종_명_스포츠_및_오락_서비스업', '업종_명_식료품_도소매업', '업종_명_음식점_및_주점업', '업종_명_제조업',
       #'업종_명_중고상품_도소매업']

X = np.array([[139405.75, 22.5, 124.0, np.log(376996219.5), 98.75, 51.25, 398, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

dtest = xgb.DMatrix(X)

In [7]:
pred = xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration))
pred

array([17.061691], dtype=float32)

In [8]:
pred[0]

17.061691

In [13]:
import torch
from torch import nn

class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(11, 121),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(121, 121),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(121, 121),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(121, 14),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [19]:
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MLP().to(device)
model_dic = torch.load("MLP_model.pth", map_location=device)

model.load_state_dict(model_dic)



input = torch.tensor([10, 2, 2, 1, 10, 10, 0, 5, 86, 95, 30]).unsqueeze(0).to(device, dtype=torch.float)


model.eval()
with torch.no_grad():
    input = input.to(device, dtype=torch.float)
    pred = model(input)
    probabilities = F.softmax(pred, dim=1)

probabilities_percent = probabilities * 100
torch.set_printoptions(precision=2, sci_mode=False)
print(probabilities_percent)



tensor([[    4.56,     0.00,    87.39,     0.10,     0.85,     1.72,     1.56,
             1.28,     0.16,     0.48,     0.00,     0.00,     1.90,     0.00]])


In [24]:
probabilities_percent[0][2]

tensor(87.39)